#depth calo
X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

depth=[3*X0_ECAL, 16*X0_ECAL, 6*X0_ECAL, 1.5*Lambda_int_HCAL, 4.1*Lambda_int_HCAL, 1.8*Lambda_int_HCAL ]
#depth=[ 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]
print(depth)

# Predictions

In [1]:
import h5py
import numpy as np
import ROOT

Welcome to JupyROOT 6.16/00


## Importing Files
Importing Both
 - file containing the energy [MeV] in each cell:
 
       Calo with 6 layers (3 ECAL: [3, 16, 6]X0_E 3 HCAL: [1.5, 4.1, 2.8]lI_H ) with [64,32,32,16,16,8]^2 pixel 
       
       Dimensions: Beam distance 150 cm, Transversal length 125x125 cm^2, 
           [11.691, 62.352, 23.382, 26.157, 71.4958, 31.3884] cm  
       
          Rad. Len.: X0_E = 3.897  X0_H = 2.357
          
          Int. Len.: lI_E = 37.578 lI_H = 17.438
 
 - file containing the topocluster information. Int number for each cell associated to the topocluster


In [2]:
#Energy Calo
f = h5py.File('../File_En/Outfile_CellInformation_HomDet_2to5GeV_FineGran128_V1.h5','r')
fN= h5py.File('../File_En/Outfile_MLPred_HomDet_2to5GeV_FineGran128_V1.h5','r')

In [3]:
print(f.keys())

<KeysViewHDF5 ['MidPoint_X_Layer1', 'MidPoint_X_Layer2', 'MidPoint_X_Layer3', 'MidPoint_X_Layer4', 'MidPoint_X_Layer5', 'MidPoint_X_Layer6', 'MidPoint_Y_Layer1', 'MidPoint_Y_Layer2', 'MidPoint_Y_Layer3', 'MidPoint_Y_Layer4', 'MidPoint_Y_Layer5', 'MidPoint_Y_Layer6', 'MidPoint_Z_Layer', 'Photon1_E', 'Photon1_Phi', 'Photon1_Theta', 'Photon2_E', 'Photon2_Phi', 'Photon2_Theta', 'Pi0_Phi', 'Pi0_Theta', 'RealRes_ChargedEnergy_Layer1', 'RealRes_ChargedEnergy_Layer2', 'RealRes_ChargedEnergy_Layer3', 'RealRes_ChargedEnergy_Layer4', 'RealRes_ChargedEnergy_Layer5', 'RealRes_ChargedEnergy_Layer6', 'RealRes_NeutralEnergy_Layer1', 'RealRes_NeutralEnergy_Layer2', 'RealRes_NeutralEnergy_Layer3', 'RealRes_NeutralEnergy_Layer4', 'RealRes_NeutralEnergy_Layer5', 'RealRes_NeutralEnergy_Layer6', 'RealRes_Noise_Layer1', 'RealRes_Noise_Layer2', 'RealRes_Noise_Layer3', 'RealRes_Noise_Layer4', 'RealRes_Noise_Layer5', 'RealRes_Noise_Layer6', 'RealRes_TotalEnergy_Layer1', 'RealRes_TotalEnergy_Layer2', 'RealRes_To

In [4]:
#cluster on total
gT = h5py.File('../File_Clust/Outfile_2to5GeV_TotalTopo.h5','r')

#cluster on charge wo noise
gCh= h5py.File('../File_Clust/Outfile_2to5GeV_ChargedTopo.h5','r')

## Calorimeter parameters (X0, lambda_I, ...)

import math

TrDi=125.

X0_ECAL = 3.897
Lambda_int_HCAL = 17.438

Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16  * X0_ECAL/2
zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2

layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}

def MakeTruthTrajectory(Theta, Phi, x_indx, y_indx) : 

    x_orig, y_orig = 0., 0.
    z_orig = -1 * (Total_Calo_Length/2 + 150.)

    if(x_indx == 5 and y_indx == 5) : x_orig, y_orig   =  10.,  10.
    if(x_indx == 5 and y_indx == -5) : x_orig, y_orig  =  10., -10.
    if(x_indx == -5 and y_indx == 5) : x_orig, y_orig  = -10.,  10.
    if(x_indx == -5 and y_indx == -5) : x_orig, y_orig = -10., -10.

    layer1_x = x_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer1_y = y_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer2_x = x_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer2_y = y_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer3_x = x_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer3_y = y_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer4_x = x_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer4_y = y_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer5_x = x_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer5_y = y_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer6_x = x_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer6_y = y_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    CellIndexLayer = []

    CellIndexLayer.append(  FindCellIndex( layer1_x, layer1_y, 'layer1' ) )
    CellIndexLayer.append(  FindCellIndex( layer2_x, layer2_y, 'layer2' ) )
    CellIndexLayer.append(  FindCellIndex( layer3_x, layer3_y, 'layer3' ) )
    CellIndexLayer.append(  FindCellIndex( layer4_x, layer4_y, 'layer4' ) )
    CellIndexLayer.append(  FindCellIndex( layer5_x, layer5_y, 'layer5' ) )
    CellIndexLayer.append(  FindCellIndex( layer6_x, layer6_y, 'layer6' ) )

    return np.array(CellIndexLayer)


def FindCellIndex(tr_x, tr_y, layer_name) : 
        
    x_idx = int( (tr_x + TrDi/2.)/TrDi * layer_struct[layer_name] )
    y_idx = int( (tr_y + TrDi/2.)/TrDi * layer_struct[layer_name] )
    
    return np.array([x_idx, y_idx])




In [5]:
import math
layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}

def FindCellIndex(tr_x, tr_y, layer_name) : 
    
    add_x, add_y = 0,0
    if( int(tr_x/(1250/128)+ 64) % int( 128/ layer_struct[layer_name]) != 0) : add_x += 1
    if( int(tr_y/(1250/128)+ 64) % int( 128/ layer_struct[layer_name]) != 0) : add_y += 1
    
    cell_x_1 = np.modf( (tr_x/(1250/128)+ 64) * layer_struct[layer_name]/128 )[1].astype(int) + add_x  
    cell_y_1 = np.modf( (tr_y/(1250/128)+ 64) * layer_struct[layer_name]/128 )[1].astype(int) + add_y
    return np.array([cell_x_1, cell_y_1])

def MakeTruthTrajectory(trk_Theta,trk_Phi,trk_X_pos,trk_Y_pos):

    X0_ECAL = 3.897
    Lambda_int_HCAL = 17.438

    Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
    Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
    Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


    zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
    zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16  * X0_ECAL/2
    zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

    zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

    zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
    zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
    zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2
    z_orig = -1 * (Total_Calo_Length/2 + 150)
    trk_X_n = trk_X_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi)
    trk_Y_n = trk_Y_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi)
    
    
    x_1 = (trk_X_n + np.abs(-zpos_ECAL1+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_1 = (trk_Y_n + np.abs(-zpos_ECAL1+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_2 = (trk_X_n + np.abs(-zpos_ECAL2+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_2 = (trk_Y_n + np.abs(-zpos_ECAL2+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_3 = (trk_X_n + np.abs(-zpos_ECAL3+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_3 = (trk_Y_n + np.abs(-zpos_ECAL3+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_4 = (trk_X_n + np.abs(-zpos_HCAL1+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_4 = (trk_Y_n + np.abs(-zpos_HCAL1+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_5 = (trk_X_n + np.abs(-zpos_HCAL2+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_5 = (trk_Y_n + np.abs(-zpos_HCAL2+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_6 = (trk_X_n + np.abs(-zpos_HCAL3+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_6 = (trk_Y_n + np.abs(-zpos_HCAL3+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    
    l1_idx = FindCellIndex(x_1, y_1, 'layer1')
    l2_idx = FindCellIndex(x_2, y_2, 'layer2')
    l3_idx = FindCellIndex(x_3, y_3, 'layer3')
    l4_idx = FindCellIndex(x_4, y_4, 'layer4')
    l5_idx = FindCellIndex(x_5, y_5, 'layer5')
    l6_idx = FindCellIndex(x_6, y_6, 'layer6')
    
    return np.array([l1_idx, l2_idx, l3_idx, l4_idx, l5_idx, l6_idx]).astype(int)

Count number of cells inside the topocluster (Type) in the event (ev) in topocluster number (n)

In [6]:
def Count_Cell(Type,ev, n):
    lay=[64,32,32,16,16,8]
    
    c=0
    for l in range(6):
        if Type=="Total":
            Topo=gT["TopoClusters"+str(l+1)][ev][0]
        
        if Type=="Charged":
            Topo=gCh["TopoClusters"+str(l+1)][ev][0]
            
        for X in range(lay[l]):
            for Y in range(lay[l]):
                
                if Topo[X][Y]==n+1: c+=1
                    
    return c



Compute List of layer's volumes in unit of Radiation length X0

In [24]:
def Vol_Cells():
    
    TrLen=125.
    
    X0_CAL=[3.897,3.897,3.897,2.357,2.357,2.357]
    Lambda_int=[37.578,37.578,37.578,17.438,17.438,17.438]
    layers=[64,32,32,16,16,8]
    Depth=[3 *3.897,16 *3.897,6 *3.897,1.5 *17.438,4.1 *17.438,1.8 *17.438]
    print("Depth in cm: ",Depth)
    
    X0_tran=[TrLen/X0_CAL[i]/layers[i] for i in range(6)]
    
    X0_depth=[Depth[i]/X0_CAL[i] for i in range(6)]
    print("Pixel size in X0: ", X0_tran)

        
    V_X0=[]
    for i in range(6):
        V_X0.append(X0_tran[i]*X0_tran[i]*X0_depth[i])
        
        
    return V_X0

In [25]:
print(Vol_Cells())

Depth in cm:  [11.690999999999999, 62.352, 23.381999999999998, 26.156999999999996, 71.49579999999999, 31.388399999999997]
Pixel size in X0:  [0.501186810366949, 1.002373620733898, 1.002373620733898, 3.3145948239287226, 3.3145948239287226, 6.629189647857445]
[0.7535646566573884, 16.076046008690952, 6.028517253259107, 121.92401214091726, 333.2589665185072, 585.2352582764029]


EN_Topo(): Compute array energy with energy (KIND) per topocluster (TOPOCL) in the event

ENN(): Compute Neutral energy predicted by Neural Network within the Topocluster (Total)

In [9]:
#Compute Array with energy (KIND) per topocluster (TOPOCL) in the event     
def En_Topo(KIND, TOPOCL, ev,n):
    layers=[64,32,32,16,16,8]
    En = 0
    EnL = 0
    EnTopo = []
    for i in range(1,7):
        if TOPOCL=="Total":
            Topo = gT['TopoClusters'+str(i)][ev][0]
            Calo = f['RealRes_'+str(KIND)+'Energy_Layer'+str(i)][ev][0]
        if TOPOCL=="Charged":    
            Calo = f['RealRes_'+str(KIND)+'Energy_Layer'+str(i)][ev][0]
            Topo = gCh['TopoClusters'+str(i)][ev][0]
            if n==True:
                Calo+=f['RealRes_Noise_Layer'+str(i)][ev][0]
            
        size=layers[i-1]
            
        TopoA=np.array(Topo[0:size,0:size])
        CaloA=np.array(Calo[0:size,0:size])

        for X in range(len(TopoA)):
            for Y in range(len(TopoA)): 
                TopoI=int(TopoA[X][Y])
                if TopoI!=0:
                    if len(EnTopo)<TopoI: EnTopo.append(Calo[X][Y])
                    else: EnTopo[TopoI-1]=EnTopo[TopoI-1]+CaloA[X][Y]
                        
    return EnTopo



def ENN(n, ev):
    layers=[64,32,32,16,16,8]
    En = 0
    for i in range(1,7):
        Topo = gT['TopoClusters'+str(i)][ev][0]
        Calo = fN['RealRes_NeutralEnergyPred_Layer'+str(i)][ev][0]
        
            
        size=layers[i-1]
            
        TopoA=np.array(Topo[0:size,0:size])
        CaloA=np.array(Calo[0:size,0:size])

        for X in range(len(TopoA)):
            for Y in range(len(TopoA)): 
                TopoI=int(TopoA[X][Y])
                if TopoI==n+1:
                    En+=CaloA[X][Y]
                        
    return En




# Number of EVENTS

In [10]:
Events=6000

In [11]:
EV=160
print("Tot_Tot",En_Topo("Total","Total", EV,False))
print("Ch_Tot",En_Topo("Charged","Total", EV,False))
print("Nu_Tot",En_Topo("Neutral","Total", EV,False))
Chaa=En_Topo("Charged","Charged", EV,True)
print("Ch_Ch",Chaa)

#(Chaa[0]-Chaa[1])/(Chaa[0]+Chaa[1])
#ENN(NTot[EV], EV)

Tot_Tot [5106.979016946271]
Ch_Tot [1705.4359701304552]
Nu_Tot [3098.5595930652053]
Ch_Ch [2047.0674769721813]


# Compute LHED, NCh, fpred, PT, dist(Cl,Tr) per event

In [ ]:
layers=[64,32,32,16,16,8]
ODist=[8,4,4,2,2,1]
ODist=[10*x for x in ODist]
Vol_Cell=Vol_Cells()

X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

#depth in interaction lengths
depth=[0, 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]

Dist=[]
NCh=[]
LHED=[]
PT=[]
f_Pred=[]


l=0
k=0
fMed=0

for ev in range(Events):
    
    if ev%100==0: print("Ev: "+str(ev))
    #COMPUTE PT
    En=f["Smeared_Track_Energy"][ev]

    ptr=math.sqrt(En*En-139.57018*139.57018)
    PT.append(ptr)


    
    #COMPUTE LHED
    
    
    #lay=compute_LHED(ev, LHED)
    Dens_En=[]
    Dens_En.append(0)
        
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_pos"]   
    C4=f["Trk_Y_pos"]
    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev]) 
    
    
    for lay in range(6):
        Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
        CCN =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]

        En_Lay=0
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                if int(CCN[X][Y])!=0 :
                    ga_we=math.exp(-((Traj_Mark[lay][0]-X)**2+(Traj_Mark[lay][1]-Y)**2)/(0.00236672*layers[lay]**2))
                    #print(X,Tr[lay][0],Y, Tr[lay][1],layers[lay], ga_we, Lay_Ev[X][Y] )
                    En_Lay+=(Lay_Ev[X][Y])*ga_we
        #print("En",En_Lay, Vol_Cell[lay])


        Dens_En.append(En_Lay/Vol_Cell[lay])
    #print("Rho",Dens_En)
    Delta_Dens=[]
    for i in range(1,7):
        Delta_Dens.append((Dens_En[i]-Dens_En[i-1])/(depth[i]-depth[i-1]))
    #print("D",depth)
    #print("DRho",Delta_Dens)

    ind=Delta_Dens.index(max(Delta_Dens))
    #print(ind)
    if Dens_En[ind+1]<Dens_En[ind]: ind-=1
    #print(ind)

    #store in LIST
    LHED.append(ind)
    
    
    #find TOPOCLUSTER associated to the track in LHED
    
    lay = ind
    
    Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
    Topo_Ev =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]   
  

    Ch=En_Topo("Charged", "Charged", ev,True)
    NTopo=len(Ch)
    
    AverPos=[[0, 0] for i in range(NTopo)]
    E_Topo_Lay=[0 for i in range(NTopo)]
    for X in range(layers[lay]):
        for Y in range(layers[lay]):
            TopoInt=int(Topo_Ev[X][Y])
            if TopoInt!=0:
                #find Center Of Topocluster (AverPos) and
                
                AverPos[TopoInt-1][0]=AverPos[TopoInt-1][0]+Lay_Ev[X][Y]*X
                AverPos[TopoInt-1][1]=AverPos[TopoInt-1][1]+Lay_Ev[X][Y]*Y
                E_Topo_Lay[TopoInt-1]=E_Topo_Lay[TopoInt-1]+Lay_Ev[X][Y]

    #print(AverPos,E_Topo_Lay, ev, lay)
    
    if E_Topo_Lay==[0]*NTopo:
        l+=1
        f_Pred.append(-1)
        NCh.append(-1)
        #print("Skipping ev: ", ev)
        continue
    
    #print(AverPos)
    #print(E_Topo_Lay)
    for j,r in enumerate(AverPos):
        E=E_Topo_Lay[j]
        if E<0.0001: continue
        for i in range(2):
            r[i]=int(r[i]/E_Topo_Lay[j]+0.5)

    #     AverPos Contains mean position of each topocluster
    #print("Center for each clust.: ",AverPos, "", lay+1)
        
        
    #print("Real Position of Track: ",Traj_Mark[lay])

        #compute distances Clus-Track
    D=[]
    for r in AverPos:
        D.append(math.sqrt((r[0]-Traj_Mark[lay][0])**2+(r[1]-Traj_Mark[lay][1])**2))
    #print("Distances Clus-Track: ",D)

        #return which clust is nearest
    MinDist=min(D)
    Dist.append(MinDist)
    
    #print(MinDist, ODist[lay])
    if MinDist > ODist[lay]:
        k+=1
        f_Pred.append(-2)
        NCh.append(-2)
        continue
        
    CN=D.index(MinDist)  
    #print("Cluster nearest to track: ",CN)
        
        #fill a list for each event with number of cluster nearest to track

    #store in LIST
    NCh.append(CN)

    
    NTop = CN
    
    fp=Ch[NTop]/ptr
    
    fMed+=fp
    
    f_Pred.append(fp)
    
    
fMed/=(Events-l-k)
print("too far",k,"no clus", l, "fMed",fMed)



Depth in cm:  [11.690999999999999, 62.352, 23.381999999999998, 26.156999999999996, 71.49579999999999, 31.388399999999997]
Pixel size in X0:  [0.501186810366949, 1.002373620733898, 1.002373620733898, 3.3145948239287226, 3.3145948239287226, 6.629189647857445]
Ev: 0
Ev: 100
Ev: 200
Ev: 300
Ev: 400
Ev: 500
Ev: 600
Ev: 700
Ev: 800
Ev: 900
Ev: 1000
Ev: 1100
Ev: 1200
Ev: 1300
Ev: 1400
Ev: 1500
Ev: 1600
Ev: 1700
Ev: 1800
Ev: 1900
Ev: 2000
Ev: 2100
Ev: 2200
Ev: 2300
Ev: 2400
Ev: 2500


## DR

In [12]:
DR1=[]
DR2=[]
DR3=[]
DR4=[]
DR5=[]
DR6=[]
DR=[DR1,DR2,DR3,DR4,DR5,DR6]

Dx1=[]
Dx2=[]
Dx3=[]
Dx4=[]
Dx5=[]
Dx6=[]
Dx=[Dx1,Dx2,Dx3,Dx4,Dx5,Dx6]


Dy1=[]
Dy2=[]
Dy3=[]
Dy4=[]
Dy5=[]
Dy6=[]
Dy=[Dy1,Dy2,Dy3,Dy4,Dy5,Dy6]

layers=[64,32,32,16,16,8]
l=[0,0,0,0,0,0]

c=[]
for ev in range(1000):
    
    if ev%100==0: print("Ev: "+str(ev))
        
    Ch=En_Topo("Charged", "Charged", ev,True)
    if len(Ch)==0:
        for i in range(6):
            DR[i].append(-15)
            Dx[i].append(-15)
            Dy[i].append(-15)
        continue

    ind=Ch.index(max(Ch))
    #NTopo=len(Ch)
    
    #C=NCh[ev]
    
    Ech=Ch[ind]

    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_pos"]   
    C4=f["Trk_Y_pos"]
    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev])
        
    AverPos=[[0, 0] for i in range(6)]
    E_Topo_Lay=[0 for i in range(6)]
    for lay in range(6):
            
        Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        Topo_Ev =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]   
        
        
        
        
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                TopoInt=int(Topo_Ev[X][Y])
                if TopoInt==ind+1:
                    #find Center Of Topocluster (AverPos) and
                    AverPos[lay][0]=AverPos[lay][0]+Lay_Ev[X][Y]*X
                    AverPos[lay][1]=AverPos[lay][1]+Lay_Ev[X][Y]*Y
                    E_Topo_Lay[lay]=E_Topo_Lay[lay]+Lay_Ev[X][Y]

        E=E_Topo_Lay[lay]
        #print(ev, lay, Ch, E)
        
        if E<0.00001:
            l[lay]+=1
            Dx[lay].append(-15)
            Dy[lay].append(-15)            
            DR[lay].append(-15)
            continue
            
        for i in range(2):
            AverPos[lay][i]=int(AverPos[lay][i]/E+0.5)
            
        Dx[lay].append(AverPos[lay][0]-Traj_Mark[lay][0])
        Dy[lay].append(AverPos[lay][1]-Traj_Mark[lay][1])
        d=math.sqrt((AverPos[lay][0]-Traj_Mark[lay][0])**2+(AverPos[lay][1]-Traj_Mark[lay][1])**2)
        if lay==1 and d>10: c.append(ev)
        
        DR[lay].append(d)



    

Ev: 0
Ev: 100
Ev: 200
Ev: 300
Ev: 400
Ev: 500
Ev: 600
Ev: 700
Ev: 800
Ev: 900


In [36]:
print(c)

[35, 248, 355, 361, 480, 487, 506, 579, 714, 810, 856, 940]


In [20]:
maxi=[25,15,15,8,8,6]
lay=1
h_dr.Delete()
h_dr=ROOT.TH1D("dr","dr",2*maxi[lay],-maxi[lay],maxi[lay])#,2*max[lay],-max[lay],max[lay])


for ev in range(1000):
    #print(lay, ev)
    h_dr.Fill(Dx[lay][ev])#,Dy[lay][ev])
can=ROOT.TCanvas( "dr","dr",1600,900);
h_dr.Draw("colz")


can.Print("../DATA_PROCESSING/2to5Pred/Pl6000/Dx"+str(lay)+".png")    
can.Close()



Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/Pl6000/Dx1.png has been created


## in PI+ + PI0 sample (Total)

layers=[64,32,32,16,16,8]

X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

depth=[0, 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]

LHEDT=[]
for ev in range(Events):
    if ev%100==0: print("ev",ev)
    Vol_Cell=Vol_Cells()
    Dens_En=[]
    Dens_En.append(0)
        
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Tr=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev]) 
    
    for lay in range(6):
        Lay_Ev=f["RealRes_TotalEnergy_Layer"+str(lay+1)][ev][0]
        
        CCN =gT["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]

        En_Lay=0
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                if int(CCN[X][Y])!=0 :
                    ga_we=math.exp(-((Tr[lay][0]-X)**2+(Tr[lay][1]-Y)**2)/(0.00236672*layers[lay]**2))
                    #print(X,Tr[lay][0],Y, Tr[lay][1],layers[lay], ga_we, Lay_Ev[X][Y] )
                    En_Lay+=Lay_Ev[X][Y]/Vol_Cell[lay]*ga_we
        
        Dens_En.append(En_Lay)

    #print("Rho",Dens_En)
    Delta_Dens=[]
    for i in range(1,7):
        Delta_Dens.append((Dens_En[i]-Dens_En[i-1])/(depth[i]-depth[i-1]))
    #print("D",depth)
    #print("DRho",Delta_Dens)

    ind=Delta_Dens.index(max(Delta_Dens))
    #print(ind)
    if Dens_En[ind+1]<Dens_En[ind]: ind-=1
    #print(ind)
    LHEDT.append(ind)




## Print LHED and fPred on png file

In [18]:
#h_LHED.Delete()
#h_Epred.Delete()
#h_Epred_prof.Delete()
h_LHED=ROOT.TH1D("LHED","LHED",6,0,6)
h_Epred=ROOT.TH3D("h_Epred","h_Epred",50,1.5e3,5.5e3,6,0,6,50,0.,2.)
h_Epred_prof= ROOT.TProfile2D("h_Epred_prof","h_Epred_prof",50,1.5e3,5.5e3,6,0,6,0.,2.)

for ev in range(Events):

    lay= LHED[ev]
    fp=f_Pred[ev]
    ptr=PT[ev]
    if fp<0: continue
    h_LHED.Fill(lay)
    
    h_Epred.Fill(ptr, lay, fp)
    h_Epred_prof.Fill(ptr, lay, fp)

f_o=ROOT.TFile("fpred.root","recreate")
h_Epred.Write()
h_Epred_prof.Write()
h_LHED.Write()
f_o.Close()

In [19]:
h_LHED.GetXaxis().SetTitle("LHED")
can=ROOT.TCanvas( "LHED","LHED",1600,900);
h_LHED.Draw("")
#ROOT.gStyle.SetOptStat(1)
can.Print("../DATA_PROCESSING/2to5Pred/Pl6000/LHED.png")    
can.Close()

h_Epred_prof.GetXaxis().SetTitle("PT (MeV)")
h_Epred_prof.GetYaxis().SetTitle("LHED")
can=ROOT.TCanvas( "Param","Param",1600,900);
ROOT.gStyle.SetOptStat(0)
h_Epred_prof.Draw("colz")
can.Print("../DATA_PROCESSING/2to5Pred/Pl6000/Param.png")    
can.Close()

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/Pl6000/LHED.png has been created
Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/Pl6000/Param.png has been created


#LHED TOTAL
hist1=ROOT.TH1D("LHEDT","LHEDT",6,0,6)
for i in range(Events):
    hist1.Fill(LHEDT[i])
hist1.GetXaxis().SetTitle("LHEDT")
can=ROOT.TCanvas( "LHEDT","LHEDT",1600,900);
hist1.Draw("")
can.Print("../DATA_PROCESSING/2to5Pred/Pl6000/LHEDT.png")    

## Compute num topocluster of interaction for total clusters

In [19]:
NTot=[]
layers=[64,32,32,16,16,8]
ODist=[8,4,4,2,2,1]
ODist=[10*x for x in ODist]
l=0
k=0

En=[]
Ech=[]
Et=[]

for ev in range(Events):
    if ev%100==0: print("Ev: "+str(ev))
    
    
    
    #define layer of first interaction
    lay=LHED[ev]
    
    Lay_Ev=f["RealRes_TotalEnergy_Layer"+str(lay+1)][ev][0]
        
    Topo_Ev =gT["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]   
  
    Charged=En_Topo("Charged", "Total", ev,False)
    Neutral=En_Topo("Neutral", "Total", ev,False)

    Total=En_Topo("Total", "Total", ev,False)
    
    NTopo=len(Total)
    
    AverPos=[[0, 0] for i in range(NTopo)]
    E_Topo_Lay=[0 for i in range(NTopo)]
    for X in range(layers[lay]):
        for Y in range(layers[lay]):
            TopoInt=int(Topo_Ev[X][Y])
            if TopoInt!=0:
                #find Center Of Topocluster (AverPos) and
                 
                AverPos[TopoInt-1][0]=AverPos[TopoInt-1][0]+Lay_Ev[X][Y]*X
                AverPos[TopoInt-1][1]=AverPos[TopoInt-1][1]+Lay_Ev[X][Y]*Y
                E_Topo_Lay[TopoInt-1]=E_Topo_Lay[TopoInt-1]+Lay_Ev[X][Y]

    #print(AverPos,E_Topo_Lay, ev, lay)
    
    if E_Topo_Lay==[0]*NTopo:
        l+=1
        CN=-1
        Et.append(-1)
        Ech.append(-1)
        En.append(-1)
        NTot.append(-1)
        #print("Skipping ev: ", ev)
        continue
    
    #print(AverPos)
    #print(E_Topo_Lay)
    for j,r in enumerate(AverPos):
        E=E_Topo_Lay[j]
        if E<0.0001: continue
        for i in range(2):
            r[i]=int(r[i]/E_Topo_Lay[j]+0.5)

    #     AverPos Contains mean position of each topocluster
    #print("Center for each clust.: ",AverPos, "", lay+1)
        
        
    #defining track position
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev])        
    #print("Real Position of Track: ",Traj_Mark[lay])

        #compute distances Clus-Track
    D=[]
    for r in AverPos:
        D.append(math.sqrt((r[0]-Traj_Mark[lay][0])**2+(r[1]-Traj_Mark[lay][1])**2))
    #print("Distances Clus-Track: ",D)

        #return which clust is nearest
    MinDist=min(D)
    
    #print(MinDist, ODist[lay])
    if MinDist > ODist[lay]:
        k+=1
        CN=-2
        Et.append(-2)
        Ech.append(-2)
        En.append(-2)
        NTot.append(-2)
        continue
        
    CN=D.index(MinDist)  
    #print("Cluster nearest to track: ",CN)
        
        #fill a list for each event with number of cluster nearest to track

    NTot.append(CN)
    
    Ech.append(Charged[CN])
    En.append(Neutral[CN])
    Et.append(Total[CN])

print("too far",k,"no clus", l)

Ev: 0
Ev: 100
Ev: 200
Ev: 300
Ev: 400
Ev: 500
Ev: 600
Ev: 700
Ev: 800
Ev: 900
Ev: 1000
Ev: 1100
Ev: 1200
Ev: 1300
Ev: 1400
Ev: 1500
Ev: 1600
Ev: 1700
Ev: 1800
Ev: 1900
Ev: 2000
Ev: 2100
Ev: 2200
Ev: 2300
Ev: 2400
Ev: 2500
Ev: 2600
Ev: 2700
Ev: 2800
Ev: 2900
Ev: 3000
Ev: 3100
Ev: 3200
Ev: 3300
Ev: 3400
Ev: 3500
Ev: 3600
Ev: 3700
Ev: 3800
Ev: 3900
Ev: 4000
Ev: 4100
Ev: 4200
Ev: 4300
Ev: 4400
Ev: 4500
Ev: 4600
Ev: 4700
Ev: 4800
Ev: 4900
Ev: 5000
Ev: 5100
Ev: 5200
Ev: 5300
Ev: 5400
Ev: 5500
Ev: 5600
Ev: 5700
Ev: 5800
Ev: 5900
too far 0 no clus 0


## without Use the track information

NTot=[]

for ev in range(Events):    
    if ev%100==0: print("Ev: "+str(ev))

    A=En_Topo("Total","Total",ev,False)
    if len(A)==0: 
        NTot.append(-1)
        continue
    
    ind=A.index(max(A))
    A.sort()
    if len(A)>1 and (A[-1]-A[-2])/(A[-1]+A[-2])<0.7  : 
        NTot.append(-2)
        continue

    NTot.append(ind)

# Results
## Plots of comparison

In [21]:
Distr=[]
String="(E_Pred(L,PT) - E-T):E-T"
try:
    h_pl
except:
    h_pl = None
else:
    if h_pl == None:
        h_pl=ROOT.TH1D(String ,String,100,-1,1)
    else:
        h_pl.Delete()
        h_pl=ROOT.TH1D(String ,String,100,-1,1) 
c=0
t=0
E=0
for ev in range(Events):
    if ev%100==0: print("Ev: ",ev)
    
    if NTot[ev]<0:
        t+=1
        continue
    if NCh[ev]<0:
        c+=1
        continue

    #Total topo
    if En[ev]==0.0:
        E+=1
        continue

    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    #NumCell
    
    #Total
    #CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    #CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(PT[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    
    k+=1
    h_pl.Fill( ( (Et[ev]-EPR*PT[ev]) - En[ev]) /(En[ev])    )    
    
print(fMed, t, c, E, Events-c-t-E)









    #h_EN.Fill(fp - E_ChN/P_Track[ev])
    #h_EN.Fill(EPR - E_ChN/P_Track[ev])
    #h_EN.Fill(fp - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill(EPR - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill( ( E_ChN-(E_Tot-E_N) ) / ( E_ChN+(E_Tot-E_N) ) )
    #h_EN.Fill((E_ChC-E_Ch)/(E_ChC+E_Ch))
    #h_EN.Fill(( ((E_ChN-E_ChC) - (E_Tot-E_Ch-E_N)) / ((E_ChN-E_ChC) + (E_Tot-E_Ch-E_N)) ))
    #h_EN.Fill((CellC-CellT)/(CellC+CellT))
    #h_EN.Fill( (ENeNe- E_N)/E_N )



Ev:  0
Ev:  100
Ev:  200
Ev:  300
Ev:  400
Ev:  500
Ev:  600
Ev:  700
Ev:  800
Ev:  900
Ev:  1000
Ev:  1100
Ev:  1200
Ev:  1300
Ev:  1400
Ev:  1500
Ev:  1600
Ev:  1700
Ev:  1800
Ev:  1900
Ev:  2000
Ev:  2100
Ev:  2200
Ev:  2300
Ev:  2400
Ev:  2500
Ev:  2600
Ev:  2700
Ev:  2800
Ev:  2900
Ev:  3000
Ev:  3100
Ev:  3200
Ev:  3300
Ev:  3400
Ev:  3500
Ev:  3600
Ev:  3700
Ev:  3800
Ev:  3900
Ev:  4000
Ev:  4100
Ev:  4200
Ev:  4300
Ev:  4400
Ev:  4500
Ev:  4600
Ev:  4700
Ev:  4800
Ev:  4900
Ev:  5000
Ev:  5100
Ev:  5200
Ev:  5300
Ev:  5400
Ev:  5500
Ev:  5600
Ev:  5700
Ev:  5800
Ev:  5900
0.5907460719075477 0 75 230 5695


In [22]:
h_pl.GetXaxis().SetTitle(String)
can=ROOT.TCanvas(String,String,1600,900);

total = ROOT.TF1("ms","gaus(0)+gaus(3)",-1,1);
total.SetParameters(6000, 0.2, 0.25,6000, 0.2, 0.5)

h_pl.Fit(total,"R");

h_pl.Draw("")
total.Draw("Same")
ROOT.gStyle.SetOptStat(1)
#can5.SetLogy()

can.Close()

 FCN=79.339 FROM MIGRAD    STATUS=CONVERGED     988 CALLS         989 TOTAL
                     EDM=1.4849e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.35099e+02   1.93809e+01   1.19603e-02  -4.40492e-05
   2  p1           2.05168e-01   6.05736e-03   2.08261e-05   4.63017e-02
   3  p2           2.07950e-01   1.68805e-02   1.41669e-05   3.41501e-02
   4  p3           4.08711e+01   2.10088e+01   6.16091e-03   5.77937e-05
   5  p4           2.28033e-01   1.73342e-02   4.87979e-05  -1.82500e-02
   6  p5           3.84543e-01   5.07585e-02   2.99522e-05   3.27186e-02


In [23]:
g=ROOT.TF1("M","gaus(0)", -1,1)
g.SetParameters(135,0.205, 0.2079)
g.Draw("Same")

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [24]:

can.Print("../DATA_PROCESSING/2to5Pred/Pl6000/"+String+"2.png")


Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/Pl6000/(E_Pred(L,PT) - E-T):E-T2.png has been created


In [53]:
Distr=[]
#String="(E_Pred(L,PT) - E-T):E-T"
String="(ENeNe - E-T):E-T"
h_EN= ROOT.TH1D(String ,String,100,-1,1)
h_EN1= ROOT.TH1D("A" ,"A",100,-1,1)
t=0
E=0
for ev in range(Events):
    if ev%100==0: print("loop ev: ",ev)
    
    

    
    if NTot[ev]<0:
        t+=1
        continue
    #if NCh[ev]<0:
    #    t+=1
    #    continue


    #Total Topo

    #Neutral
    A=En_Topo("Neutral","Total",ev,False)
    E_N=A[NTot[ev]]

    if E_N==0.0:
        E+=1
        continue

    #Total
    #A=En_Topo("Total","Total",ev, False)
    #E_Tot=A[NTot[ev]]
    
    #Charged
    #A=En_Topo("Charged","Total",ev, False)
    #E_Ch=A[NTot[ev]]
    
    
    #Neural Network
    ENeNe=ENN(NTot[ev], ev)
    

    
    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    
    
    
    #NumCell
    
    #Total
    #CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    #CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    #EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(P_Track[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    

    #h_EN.Fill(fp - E_ChN/P_Track[ev])
    #h_EN.Fill(EPR - E_ChN/P_Track[ev])
    #h_EN.Fill(fp - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill(EPR - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill( ( E_ChN-(E_Tot-E_N) ) / ( E_ChN+(E_Tot-E_N) ) )
    #h_EN.Fill((E_ChC-E_Ch)/(E_ChC+E_Ch))
    #h_EN.Fill(( ((E_ChN-E_ChC) - (E_Tot-E_Ch-E_N)) / ((E_ChN-E_ChC) + (E_Tot-E_Ch-E_N)) ))
    #h_EN.Fill((CellC-CellT)/(CellC+CellT))
    
    #h_EN.Fill( ( (E_Tot-EPR*P_Track[ev]) - E_N) /(E_N)    )    
    #h_EN.Fill( (ENeNe- E_N)/E_N )
    
    #if TotNN[ev]<12000 or TotNN[ev]>19000:
    #    continue

    h_EN1.Fill( (ENeNe- E_N)/E_N )


    
print(fMed, "out Range ENN:","En=0:",  t, E, Events-E-t)

loop ev:  0
loop ev:  100
loop ev:  200
loop ev:  300
loop ev:  400
loop ev:  500


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/lox/opt/anaconda3/envs/piton3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-2f27cd62a875>", line 42, in <module>
    ENeNe=ENN(NTot[ev], ev)
  File "<ipython-input-9-f1365023546f>", line 48, in ENN
    TopoI=int(TopoA[X][Y])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lox/opt/anaconda3/envs/piton3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lox/opt/anaconda3/envs/piton3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_record

KeyboardInterrupt: 

In [46]:
h_EN1.GetXaxis().SetTitle(String)
can5=ROOT.TCanvas(String,String,1600,900);
h_EN1.SetLineColor(1);

total2 = ROOT.TF1("ms","gaus(0)+gaus(3)",-1,1);
total2.SetParameters(6000, -0.35, 0.12,6000, -0.035, 0.3)

h_EN1.Fit(total2,"R");

h_EN1.Draw("")
total2.Draw("Same")



 FCN=299.908 FROM MIGRAD    STATUS=CONVERGED     720 CALLS         721 TOTAL
                     EDM=3.53272e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -8.38006e+01   1.75457e+01   2.39020e-02   1.34516e-05
   2  p1          -2.42117e-01   9.53171e-03   3.25564e-05   1.08157e-02
   3  p2          -1.02343e-01   1.12793e-02   2.32006e-05   6.33836e-03
   4  p3           3.16747e+02   1.34035e+01   2.91618e-02   1.39785e-05
   5  p4          -7.35487e-02   8.05698e-03   1.39997e-05   8.06993e-06
   6  p5           1.63957e-01   3.59258e-03   8.61397e-06   4.71633e-02


Warning in <TCanvas::Constructor>: Deleting canvas with same name: (ENeNe - E-T):E-T


In [44]:

#ROOT.gStyle.SetOptStat(1)
can5.Print("../DATA_PROCESSING/2to5Pred/Pl6000/"+String+"3.png")

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/Pl6000/(ENeNe - E-T):E-T3.png has been created


Distr=[]
String="(E_Pred(L,PT)-E-T):E-T_vs_LFI "
h= ROOT.TH2D(String ,String,100,-1,1,100,-1,1)
k=0
for ev in range(Events):
    if ev%100==0: print("loop ev: ",ev)
    
    if NTot[ev]<0:
        continue
    if NCh[ev]<0:
        continue
    
    #Total Topo
    
    #Neutral 
    A=En_Topo("Neutral","Total",ev,False)
    E_N=A[NTot[ev]]
    
    #Total
    A=En_Topo("Total","Total",ev, False)
    E_Tot=A[NTot[ev]]
    
    #Charged
    #A=En_Topo("Charged","Total",ev, False)
    #E_Ch=A[NTot[ev]]
    
  
    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    #NumCell
    
    #Total
    CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(P_Track[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    
    if E_N==0: 
        print(ev)
        continue
    k+=1
    h.Fill( ( (E_Tot-EPR*P_Track[ev]) - E_N) /(E_N)  ,(CellC-CellT)/(CellC+CellT))    
    #h_EN.Fill( (E_N -ENNN)/E_N )
    
print(fp, k)

f_o=ROOT.TFile("Corr.root","recreate")
h.Write()
f_o.Close()